In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import SelectKBest, SelectPercentile, mutual_info_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
fake = pd.read_csv('C:\\Users\\xrist\\Desktop\\Thesis\\datasets\\celebrity news\\gossipcop_fake.csv')
true = pd.read_csv('C:\\Users\\xrist\\Desktop\\Thesis\\datasets\\celebrity news\\gossipcop_real.csv')

fake['label'] = 1
true['label'] = 0

gossip = pd.concat([true,fake]).reset_index(drop=True)

In [3]:
gossip = gossip.drop(['id', 'news_url', 'tweet_ids'], axis=1)

In [4]:
gossip

,title,label
0,Teen Mom Star Jenelle Evans' Wedding Dress Is ...,0
1,Kylie Jenner refusing to discuss Tyga on Life ...,0
2,Quinn Perkins,0
3,I Tried Kim Kardashian's Butt Workout & Am For...,0
4,Celine Dion donates concert proceeds to Vegas ...,0
...,...,...
22135,September 11: Celebrities Remember 9/11 (TWEETS),1
22136,NASCAR owners threaten to fire drivers who pro...,1
22137,The 7 signs that David Beckham is definitely h...,1
22138,Ryan Gosling and Eva Mendes Did Not Get Marrie...,1


In [5]:
!pip install truecase
import truecase
#Make the capitalisation nice
gossip['title'] = gossip['title'].apply(lambda x:truecase.get_true_case(x))

In [6]:
def count_pos_tags(column, tag):
    from nltk import word_tokenize, pos_tag
    lst = []
    for idx,x in enumerate(column):
        lst.append(sum(1 for word, pos in pos_tag(word_tokenize(x)) if pos.startswith(tag)))
    return lst

In [7]:
vcount = count_pos_tags(gossip.title, 'VB')
gossip['verb_count'] = vcount

In [8]:
#Number of proper nouns
pncount = count_pos_tags(gossip.title, 'NNP')
gossip['proper_noun_count'] = pncount

In [9]:
#Number of total nouns (including proper_nouns)
ncount = count_pos_tags(gossip.title, 'NN')

#Number of nouns
nouns = abs(np.subtract(np.array(pncount), np.array(ncount)))
gossip['noun_count'] = nouns

In [10]:
#Number of adjectives
jcount = count_pos_tags(gossip.title, 'JJ')
gossip['adjective_count'] = jcount

#Number of adverbs
acount = count_pos_tags(gossip.title, 'RB')
gossip['adverb_count'] = acount



In [11]:
#emotiveness
gossip['emotiveness'] = (gossip['adjective_count']+ gossip['adverb_count']) / (gossip['noun_count'] + gossip['verb_count'])

In [12]:
!pip install vaderSentiment
SA = SentimentIntensityAnalyzer()

In [13]:

#Sentiment analysis
sa = []
for idx,x in enumerate(gossip['title']):
    sa.append(SA.polarity_scores(x))
    
    
negative = pd.DataFrame([d['neg'] for d in sa])
positive = pd.DataFrame([d['pos'] for d in sa])
neutral = pd.DataFrame([d['neu'] for d in sa])
compound = pd.DataFrame([d['compound'] for d in sa])
#Attach sentiment labels
gossip['compound'] = compound
gossip['neutral'] = neutral

In [14]:
gp = gossip.loc[:,['verb_count','proper_noun_count', 'emotiveness', 'compound', 'neutral', 'label']]

In [15]:
gp

,verb_count,proper_noun_count,emotiveness,compound,neutral,label
0,1,3,0.75,0.0000,1.000,0
1,2,4,0.00,-0.4019,0.769,0
2,0,0,0.00,0.0000,1.000,0
3,3,2,0.20,0.0000,1.000,0
4,3,2,0.20,-0.3182,0.777,0
...,...,...,...,...,...,...
22135,1,1,0.00,0.0000,1.000,1
22136,3,0,0.00,-0.7184,0.417,1
22137,2,3,0.25,-0.3400,0.595,1
22138,3,3,1.00,0.0000,1.000,1


In [16]:
gp.loc[pd.isna(gp).any(1), :].index

Int64Index([   36,    41,    64,    87,   278,   318,   398,   408,   416,
              472,
            ...
            21879, 21893, 21908, 21922, 21976, 21998, 22038, 22050, 22088,
            22121],
           dtype='int64', length=461)

In [17]:
gp.replace([np.inf, -np.inf], np.nan, inplace=True)

nan = gp.loc[pd.isna(gp).any(1), :].index

for x in nan:
    gp = gp.drop(index = x, axis = 1)
    
gp.loc[pd.isna(gp).any(1), :].index

Int64Index([], dtype='int64')

In [18]:
gp.to_csv("gossip news for cv", index = False)

In [19]:
len(nan)

518

In [21]:
gossip.shape

(22140, 10)